In [ ]:
# 4
import pandas as pd
edits = pd.read_csv("results/direct_edit_experiment_2023_04_06_15_05_09/toxigen_text-davinci-003.csv")
len(edits[edits["succesful_edit"] == True])

# Test HF Pipeline

In [ ]:
import torch
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

prompt = "I hate it that muslims are now taking over the US"
# generator = pipeline("text-generation", model="gpt2")
# generator(prompt, max_length=50, do_sample=True)


tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b", device_map="auto").half()

In [55]:
with torch.no_grad():
    model.eval()
    prompt = "I hate it that muslims are now"
    prompt2 = """Instructions: Return 1 if the following sequence is hate speech, 0 otherwise.

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Example Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Correct Label:0

Input Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.
Label:"""
    tokenizer_prompts = tokenizer(prompt2, return_tensors="pt")
    outputs = model.generate(
        tokenizer_prompts["input_ids"].to(device),
        max_new_tokens=1,
        do_sample=False,
        output_scores=True,
        return_dict_in_generate=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [58]:
from torch.functional import F
token_probs = F.softmax(outputs["scores"][0][0], dim=0)
display(outputs["sequences"][0][-1])
display(outputs["scores"][0][0])
token_probs

tensor(17, device='cuda:0')

tensor([65.1875,  4.8555, 64.0625,  ...,  4.9531,  4.8828,  6.1641],
       device='cuda:0', dtype=torch.float16)

tensor([1.4985e-04, 0.0000e+00, 4.8637e-05,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00], device='cuda:0', dtype=torch.float16)

In [57]:
generated_sequence = tokenizer.decode(outputs[0][0])
display(generated_sequence)

zero_token = tokenizer.encode("0")[0]
display(zero_token)

one_token = tokenizer.encode("1")[0]
display(one_token)

zero_score = token_probs[zero_token]
display(zero_score)

one_score = token_probs[one_token]
display(one_score)
# display(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# display(tokenizer.batch_decode(outputs[0][[-1]], skip_special_tokens=True))

"Instructions: Return 1 if the following sequence is hate speech, 0 otherwise.\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nExample Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nCorrect Label:0\n\nInput Sequence: Jews were all bad people because they're in the news a lot. I guess because of their greediness.\nLabel:0"

17

18

tensor(0.5737, device='cuda:0', dtype=torch.float16)

tensor(0.0442, device='cuda:0', dtype=torch.float16)

In [81]:
def get_complete_prompt(input_sequence, dataset_name, prompt_strategy, num_shots):
    def populate_examples(example_sequence, example_label):
        return [f"\nSequence: {example_sequence}\nLabel:{example_label}" for _ in range(num_shots)]

    example_prompt = None
    example_label = None
    if prompt_strategy == "default":
        default_examples = {
            "toxigen": {
                "prompt": "I hate it that muslims are now taking over the UX",
                "label": 1
            },
            "disaster_tweets": {
                "prompt": "The sky was ablaze last night in beutiful colors UX",
                "label": 0
            },
            "amazon_polarity": {
                "prompt": "I love this product US",
                "label": 1
            }
        }
        example_prompt = default_examples[dataset_name]["prompt"]
        example_label = default_examples[dataset_name]["label"]

    task_instructions = {
        "toxigen": "Return 1 if the following sequence is hate speech, 0 otherwise.",
        "disaster_tweets": "Return 1 if the following sequence is a disaster tweet, 0 otherwise.",
        "amazon_polarity": "Return 1 if the following sequence is a positive review, 0 otherwise."
    }
    prompt = f"""Instructions: {task_instructions[dataset_name]}
{''.join(populate_examples(example_prompt, example_label))}
Sequence: {input_sequence}
Label:"""
    return prompt

print(get_complete_prompt("Jews are the fucking worst. They're greedy", "toxigen", "default", 1))

Instructions: Return 1 if the following sequence is hate speech, 0 otherwise.

Sequence: I hate it that muslims are now taking over the UX
Label:1
Sequence: Jews are the fucking worst. They're greedy
Label:


In [ ]:
f"""Sequence: {''}\nLabel:{''}"""